In [1]:
import torch
import pandas as pd
import numpy as np

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [2]:
# tokenizer import 
tokenizer = BertTokenizer.from_pretrained('klue/bert-base', do_lower_case=False)

In [3]:
# bert model 불러온 후 저장된 모델 상태 적용
model2 =  BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5)
# model2.load_state_dict(torch.load('bert_model_ver2_st_0505.pt'))
model2.load_state_dict(torch.load('bert_model_ver5(93)_st_0505.pt'))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

<All keys matched successfully>

In [4]:
# device 적용
device = torch.device("cuda:0")
model2.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [5]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [6]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model2.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model2(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits


In [10]:
np.argmax((test_sentences([''])))

3

In [7]:
sentences = pd.read_csv('review_test_data_total_ver2.csv', encoding = 'utf-8')
sentences_re = []
for i in sentences['review']:
    # print(i)
    # print('---------------------')
    logits = test_sentences([i])
    # print(logits)
    # print(np.argmax(logits))
    if np.argmax(logits) == 0 :
        print(i)
        print("⭐")
        print('-----------------------------------------------------------------------------------------------')
        sentences_re.append(1)
    elif np.argmax(logits) == 1 :
        print(i)
        print("⭐⭐")
        print('-----------------------------------------------------------------------------------------------')
        sentences_re.append(2)
    elif np.argmax(logits) == 2 :
        print(i)
        print("⭐⭐⭐")
        print('-----------------------------------------------------------------------------------------------')
        sentences_re.append(3)
    elif np.argmax(logits) == 3 :
        print(i)
        print("⭐⭐⭐⭐")
        print('-----------------------------------------------------------------------------------------------')
        sentences_re.append(4)
    else :
        print(i)
        print("⭐⭐⭐⭐⭐")
        print('-----------------------------------------------------------------------------------------------')
        sentences_re.append(5)

이것저것 많이 올려놓을 수 있어서 좋고 내장된 콘센트도 좋아요 수납도 총 5칸 있어서 편해요 근데 예전 수납 침대에는 매트리스 아래 넓은 부분에 캐리어를 넣어놨는데 요 프레임은 높이(깊이)가 낮아서 캐리어 넣고 안 닫히더라구요 그점이 좀 아쉽지만 만족합니다 배송도 제가 원하는 날짜에 맞춰주셨고 기사님도 친절하고 좋았어요
⭐⭐⭐⭐⭐
-----------------------------------------------------------------------------------------------
주문한지 10 일 지나서 온것같아요 근데 제 인생에서 다시는 리샘가구 구매할일 없을꺼 같습니다 침대 사용하지도 않았는데 더럽고 마감 엉망이고 물에 뿔은것처럼 합판 다 벌어지고 서랍 열었더니 경첩 나가서 기사님이 AS 해주셨는데 못이 외관을 뚫고 나갔어요 기사님 가신 후 확인해보니 더 화가나더군요 침대인지 쓰레기인지... 기사님 다시 방문하는거 죄송해서 그냥 쓰려구요 아빠가 가구회사에서 일하는데 나무가 얇고 엉망진창이라고 합니다 엄청 싼 가격도 아닌데 말이죠., 침대 구매하려고 하는분들 돈 좀 더주더라도 좋은거 구매하시길 바래요
⭐
-----------------------------------------------------------------------------------------------
크고 침대 좋아요. 기사님이 생각한시간보다 더 빨리와서는 승질같이내서 기분이 안좋았어요ㅎ 그래도 그정도면 친절...
⭐⭐⭐⭐⭐
-----------------------------------------------------------------------------------------------
배송은 생각보다 빨리왔으나 방바닥에 찍힘이 생겼어요..ㅜ 배송은 받을수잇는 전날에 전화주셔서 3번정도 조율했어요 물건 군데군데 흠집이.약간은.있는디 엄청티나진않아요 가운데칸은 뻥뚫려잇어요 차라리 벙커칸을 좀 크게만들어주셨으면 좋앗을텐데 아쉽네요 그리고 헤드부분 윗부분이 얕아요 

In [39]:
def get_counts(seq):
    counts = {} 
    for x in seq: 
        if x in counts: 
            counts[x] += 1
        else: counts[x] = 1 
    return counts
counts = get_counts(sentences_re)

print(counts)

{5: 42, 1: 55, 4: 18, 3: 27, 2: 38}
